In [101]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras import metrics
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.layers import Flatten

from nltk import word_tokenize, pos_tag, chunk
from pprint import pprint
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

from pprint import pprint
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [97]:
#Load the entire dataset
data = pd.read_csv('../../Results/JobsDataset.csv', header = 0, names = ['Query', 'Job Title', 'Description'])

In [112]:
## Data cleaning
#Create Job description list
job_descriptions=[]
for job in data.Description:
    j = job.replace(',', '')
    job_descriptions.append(j)
    
#Words tokenization
jobs = [word_tokenize(d) for d in job_descriptions]

#Remove Capitalization
no_capitals =[]
for job in jobs:
    no_capitals.append([j.lower() for j in job])

#Lemmatize
lemmatizer = WordNetLemmatizer()
lem=[]
for job in no_capitals:
    lem.append([lemmatizer.lemmatize(j) for j in job])

#Remove stopwords
filtered_words = []
for job in lem:
    filtered_words.append([j for j in job if not j in stopwords.words('english')])

#Remove symbols
cleaned_description=[]
for job in filtered_words:
    cleaned_description.append([j for j in job if not j in ['(',')','.',',',':','%']])

#Final cleaned description list                                                            
cleaned_desc=[]
for description in cleaned_description:
    cleaned_desc.append(" ".join(description))

                                                            
#create new df 
df = pd.DataFrame({'Query':list(data.Query),'Description':cleaned_desc})


In [105]:
#Split data to train and test (80 - 20)

train, test = train_test_split(df, test_size = 0.2)

train_descs = train['Description']
train_labels = train['Query']
#train_labels = train['Job Title']
 
test_descs = test['Description']
test_labels = test['Query']
#test_labels = test['Job Title']

In [106]:
##Parameters
#Encoding
vocab_size = 1000
max_length = 1000

#Model
num_labels = 25
embedding_dimensios = 15
nb_epoch = 30

In [107]:
###Training Data
# Encode the jobs descriptions
encoded_docs = [one_hot(d, vocab_size) for d in train_descs]
# pad documents to a max length
x_train = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#Binarize the job titles
encoder = LabelBinarizer()
encoder.fit(train_labels)
y_train = encoder.transform(train_labels)


###Test Data
# Encode the jobs descriptions
encoded_docs = [one_hot(d, vocab_size) for d in test_descs]
# pad documents to a max length
x_test = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#Binarize the job titles
encoder = LabelBinarizer()
encoder.fit(test_labels)
y_test = encoder.transform(test_labels)

In [109]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dimensios, input_length=max_length))
model.add(Flatten())
model.add(Dense(num_labels))
model.add(Activation('softmax'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 1000, 15)          15000     
_________________________________________________________________
flatten_10 (Flatten)         (None, 15000)             0         
_________________________________________________________________
dense_26 (Dense)             (None, 25)                375025    
_________________________________________________________________
activation_21 (Activation)   (None, 25)                0         
Total params: 390,025
Trainable params: 390,025
Non-trainable params: 0
_________________________________________________________________
None


In [110]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam', # or 'sgd'
              metrics = [metrics.categorical_accuracy, 'accuracy'])

In [111]:
# fit the model
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_split=0.1)

Train on 7200 samples, validate on 800 samples
Epoch 1/30
7200/7200 [==============================] - 6s 811us/step - loss: 2.8559 - categorical_accuracy: 0.1976 - acc: 0.1976 - val_loss: 2.2277 - val_categorical_accuracy: 0.3863 - val_acc: 0.3863
Epoch 2/30
7200/7200 [==============================] - 5s 710us/step - loss: 1.1271 - categorical_accuracy: 0.7654 - acc: 0.7654 - val_loss: 1.9449 - val_categorical_accuracy: 0.4075 - val_acc: 0.4075
Epoch 3/30
7200/7200 [==============================] - 5s 724us/step - loss: 0.3487 - categorical_accuracy: 0.9535 - acc: 0.9535 - val_loss: 2.0453 - val_categorical_accuracy: 0.4063 - val_acc: 0.4063
Epoch 4/30
7200/7200 [==============================] - 5s 718us/step - loss: 0.1695 - categorical_accuracy: 0.9846 - acc: 0.9846 - val_loss: 2.2074 - val_categorical_accuracy: 0.3950 - val_acc: 0.3950
Epoch 5/30
7200/7200 [==============================] - 5s 714us/step - loss: 0.1350 - categorical_accuracy: 0.9876 - acc: 0.9876 - val_loss: 2.2

In [113]:
batch_size = 10
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('\nTest categorical_cro    ssentropy:', score[0])
print('Categorical accuracy:', score[1])
print('Accuracy:', score[2])

2000/2000 [==============================] - 0s 100us/step

Test categorical_crossentropy: 4.1084576666355135
Categorical accuracy: 0.32650000631809234
Accuracy: 0.32650000631809234
